In [5]:
# Imports
import cv2 
# USE PYTHON 3.11
import pandas as pd
import time

In [2]:
# Set up Haar-Casacades

# Load classifier
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Incorporate Nurses State and patient state 
# Load the Nurses Data
nurse_df = pd.read_csv("Nurse_Stress_Data(Sheet1).csv", parse_dates=["Timestamp"])

# Load the patient Data
triage_df = pd.read_csv("Nurse_Stress_Data(Sheet1).csv", parse_dates=["Timestamp"])

# Get nurse state based on time
def get_current_nurse_state(t):
    if t < len(nurse_df):
        return nurse_df.loc[t, 'Stress_State']
    else:
        return nurse_df.iloc[-1]['Stress_State']

# Get patient data at Time t
def get_current_patient_data(triage_df, t):
    if t < len(triage_df):
        return triage_df.iloc[t].copy() 
    else:
        return triage_df.iloc[-1].copy()

In [3]:
def detect_bounding_boxes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(20, 20))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
    
    return faces

In [8]:
# Start up web-cam
capture = cv2.VideoCapture(1) # 0 for laptop camera, otherwise provide path to video

# Time frame counter for real time use
t = 0  


print(" Starting live face detection and Triaging Assist")
print("Press 'q' to stop.")

# Continuously read frames and apply haar cascades
while True:
    result, frame = capture.read() # Read a frame from the video

    if not result:
        print("ERROR! Breaking")
        break 

    faces = detect_bounding_boxes(frame)
    nurse_state = get_current_nurse_state(t)
    patient_info = get_current_patient_data(triage_df, t)

    # Show current nurse emotional tier
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
        cv2.putText(frame, f"Nurse: {nurse_state}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        #Dynamic Triage Display 
        if nurse_state == "Normal":
            display_text = (
                f"Name: {patient_info.get('Name','')}, Age: {patient_info.get('Age','')}, Sex: {patient_info.get('Sex','')}, "
                f"Arrival: {patient_info.get('Arrival_Method','')} at {patient_info.get('Arrival_Time','')}, "
                f"Triage: {patient_info.get('Triage_Level','')}, Next Assessment: {patient_info.get('Next_Triage_Time','')}, "
                f"Chief Complaint: {patient_info.get('Chief_Complaint','')}, CEDIS: {patient_info.get('CEDIS','')}, "
                f"Vitals: HR:{patient_info['Patient_HR']} BP:{patient_info['BP_Systolic']}/{patient_info['BP_Diastolic']} RR:{patient_info['RR']} SpO2:{patient_info['SpO2']} Temp:{patient_info['Temp']} Pain:{patient_info['Pain']}, "
                f"Modifiers: {patient_info.get('Modifier1','')}, {patient_info.get('Modifier2','')}, {patient_info.get('Modifier3','')}, "
                f"Nurse ID: {patient_info.get('Nurse_ID','')}, Alarm: {patient_info['Alarm_Note']}"
            )
        elif nurse_state == "Moderate":
            display_text = (
                f"Name: {patient_info.get('Name','')}, Age: {patient_info.get('Age','')}, Sex: {patient_info.get('Sex','')}, "
                f"Triage: {patient_info.get('Triage_Level','')}, Next Assessment: {patient_info.get('Next_Triage_Time','')}, "
                f"CEDIS: {patient_info.get('CEDIS','')}, "
                f"Vitals: HR:{patient_info['Patient_HR']} BP:{patient_info['BP_Systolic']}/{patient_info['BP_Diastolic']} RR:{patient_info['RR']}, "
                f"Modifiers: {patient_info.get('Modifier1','')}, {patient_info.get('Modifier2','')}, Alarm: {patient_info['Alarm_Note']}"
            )
        else:  # High
            display_text = (
                f"Name: {patient_info.get('Name','')}, "
                f"Triage: {patient_info.get('Triage_Level','')}, "
                f"CEDIS: {patient_info.get('CEDIS','')}, "
                f"Vitals: HR:{patient_info['Patient_HR']} BP:{patient_info['BP_Systolic']}/{patient_info['BP_Diastolic']}, "
                f"Alarm: {patient_info['Alarm_Note']}"
            )
            
        # Overlay dynamic triage info above nurse state text
        y0 = y - 50
        for i, line in enumerate(display_text.split(", ")):
            cv2.putText(frame, line, (x, y0 + i*15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 255), 1)



    cv2.imshow("Detection", frame) # Display processed frame in a window

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break 

    # Simulate time/frame progression
    t += 1
    time.sleep(0.5)

capture.release()
cv2.destroyAllWindows()

 Starting live face detection and Triaging Assist
Press 'q' to stop.
ERROR! Breaking
